In [1]:
from langchain.chains.pipeline import Pipeline
from langchain.chains.custom import SimpleCustomChain
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain import Prompt

In [2]:
prompt_template = """You are given the below API Documentation:

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit	String	No	celsius	If fahrenheit is set, all temperature values are converted to Fahrenheit.
windspeed_unit	String	No	kmh	Other wind speed speed units: ms, mph and kn
precipitation_unit	String	No	mm	Other precipitation amount units: inch
timeformat	String	No	iso8601	If format unixtime is selected, all time values are returned in UNIX epoch time in seconds. Please note that all timestamp are in GMT+0! For daily values with unix timestamps, please apply utc_offset_seconds again to get the correct date.
timezone	String	No	GMT	If timezone is set, all timestamps are returned as local-time and data is returned starting at 00:00 local-time. Any time zone name from the time zone database is supported. If auto is set as a time zone, the coordinates will be automatically resolved to the local time zone.
past_days	Integer (0-2)	No	0	If past_days is set, yesterday or the day before yesterday data are also returned.
start_date
end_date	String (yyyy-mm-dd)	No		The time interval to get weather data. A day must be specified as an ISO8601 date (e.g. 2022-06-30).
models	String array	No	auto	Manually select one or more weather models. Per default, the best suitable weather models will be combined.

Hourly Parameter Definition
The parameter &hourly= accepts the following values. Most weather variables are given as an instantaneous value for the indicated hour. Some variables like precipitation are calculated from the preceding hour as an average or sum.

Variable	Valid time	Unit	Description
temperature_2m	Instant	°C (°F)	Air temperature at 2 meters above ground
relativehumidity_2m	Instant	%	Relative humidity at 2 meters above ground
dewpoint_2m	Instant	°C (°F)	Dew point temperature at 2 meters above ground
apparent_temperature	Instant	°C (°F)	Apparent temperature is the perceived feels-like temperature combining wind chill factor, relative humidity and solar radiation
pressure_msl
surface_pressure	Instant	hPa	Atmospheric air pressure reduced to mean sea level (msl) or pressure at surface. Typically pressure on mean sea level is used in meteorology. Surface pressure gets lower with increasing elevation.
cloudcover	Instant	%	Total cloud cover as an area fraction
cloudcover_low	Instant	%	Low level clouds and fog up to 3 km altitude
cloudcover_mid	Instant	%	Mid level clouds from 3 to 8 km altitude
cloudcover_high	Instant	%	High level clouds from 8 km altitude
windspeed_10m
windspeed_80m
windspeed_120m
windspeed_180m	Instant	km/h (mph, m/s, knots)	Wind speed at 10, 80, 120 or 180 meters above ground. Wind speed on 10 meters is the standard level.
winddirection_10m
winddirection_80m
winddirection_120m
winddirection_180m	Instant	°	Wind direction at 10, 80, 120 or 180 meters above ground
windgusts_10m	Preceding hour max	km/h (mph, m/s, knots)	Gusts at 10 meters above ground as a maximum of the preceding hour
shortwave_radiation	Preceding hour mean	W/m²	Shortwave solar radiation as average of the preceding hour. This is equal to the total global horizontal irradiation
direct_radiation
direct_normal_irradiance	Preceding hour mean	W/m²	Direct solar radiation as average of the preceding hour on the horizontal plane and the normal plane (perpendicular to the sun)
diffuse_radiation	Preceding hour mean	W/m²	Diffuse solar radiation as average of the preceding hour
vapor_pressure_deficit	Instant	kPa	Vapor Pressure Deificit (VPD) in kilopascal (kPa). For high VPD (>1.6), water transpiration of plants increases. For low VPD (<0.4), transpiration decreases
evapotranspiration	Preceding hour sum	mm (inch)	Evapotranspration from land surface and plants that weather models assumes for this location. Available soil water is considered. 1 mm evapotranspiration per hour equals 1 liter of water per spare meter.
et0_fao_evapotranspiration	Preceding hour sum	mm (inch)	ET₀ Reference Evapotranspiration of a well watered grass field. Based on FAO-56 Penman-Monteith equations ET₀ is calculated from temperature, wind speed, humidity and solar radiation. Unlimited soil water is assumed. ET₀ is commonly used to estimate the required irrigation for plants.
precipitation	Preceding hour sum	mm (inch)	Total precipitation (rain, showers, snow) sum of the preceding hour
snowfall	Preceding hour sum	cm (inch)	Snowfall amount of the preceding hour in centimeters. For the water equivalent in millimeter, divide by 7. E.g. 7 cm snow = 10 mm precipitation water equivalent
rain	Preceding hour sum	mm (inch)	Rain from large scale weather systems of the preceding hour in millimeter
showers	Preceding hour sum	mm (inch)	Showers from convective precipitation in millimeters from the preceding hour
weathercode	Instant	WMO code	Weather condition as a numeric code. Follow WMO weather interpretation codes. See table below for details.
snow_depth	Instant	meters	Snow depth on the ground
freezinglevel_height	Instant	meters	Altitude above sea level of the 0°C level
visibility	Instant	meters	Viewing distance in meters. Influenced by low clouds, humidity and aerosols. Maximum visibility is approximately 24 km.
soil_temperature_0cm
soil_temperature_6cm
soil_temperature_18cm
soil_temperature_54cm	Instant	°C (°F)	Temperature in the soil at 0, 6, 18 and 54 cm depths. 0 cm is the surface temperature on land or water surface temperature on water.
soil_moisture_0_1cm
soil_moisture_1_3cm
soil_moisture_3_9cm
soil_moisture_9_27cm
soil_moisture_27_81cm	Instant	m³/m³	Average soil water content as volumetric mixing ratio at 0-1, 1-3, 3-9, 9-27 and 27-81 cm depths.

Using that documentation, write a query that you could send to the open meteo api to answer the following question.

Question: {question}
GET Request: /v1/forecast?"""

In [3]:
prompt = Prompt(input_variables=["question"], template=prompt_template)

In [4]:
question_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt, output_key="meteo_query")

In [5]:
import requests
def make_open_meteo_request(req):
    return str(requests.get(f"https://api.open-meteo.com/v1/forecast?{req}").json())

In [6]:
custom_chain = SimpleCustomChain(func=make_open_meteo_request, input_key="meteo_query", output_key="meteo_answer")

In [7]:
answer_prompt_template = """You are given the below weather information:

{meteo_answer}

Now answer the following question:

Question: {question}
Answer:"""
answer_prompt = Prompt(input_variables=["question", "meteo_answer"], template=answer_prompt_template)

answer_chain = LLMChain(llm=OpenAI(temperature=0), prompt=answer_prompt)

In [8]:
pipeline = Pipeline(chains=[question_chain, custom_chain, answer_chain], input_variables=["question"], verbose=True)

In [9]:
question = "is it snowing in boston?"

In [10]:
pipeline.run(question)



> Entering new chain...
Chain 0:
{'meteo_query': 'latitude=42.3601&longitude=-71.0589&hourly=snowfall'}

Chain 1:
{'meteo_answer': "{'latitude': 42.36515, 'longitude': -71.0618, 'generationtime_ms': 0.4010200500488281, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 9.0, 'hourly_units': {'time': 'iso8601', 'snowfall': 'cm'}, 'hourly': {'time': ['2022-11-19T00:00', '2022-11-19T01:00', '2022-11-19T02:00', '2022-11-19T03:00', '2022-11-19T04:00', '2022-11-19T05:00', '2022-11-19T06:00', '2022-11-19T07:00', '2022-11-19T08:00', '2022-11-19T09:00', '2022-11-19T10:00', '2022-11-19T11:00', '2022-11-19T12:00', '2022-11-19T13:00', '2022-11-19T14:00', '2022-11-19T15:00', '2022-11-19T16:00', '2022-11-19T17:00', '2022-11-19T18:00', '2022-11-19T19:00', '2022-11-19T20:00', '2022-11-19T21:00', '2022-11-19T22:00', '2022-11-19T23:00', '2022-11-20T00:00', '2022-11-20T01:00', '2022-11-20T02:00', '2022-11-20T03:00', '2022-11-20T04:00', '2022-11-20T05:00', '2022-11-2

' No'